In [1]:
import pandas as pd
import numpy as np
from typing import List

In [2]:
pd.set_option('mode.chained_assignment',  None) 
pd.options.display.float_format = '{:.3f}'.format

소수점 표시를 위한 출력 설정 변경

# 데이터 불러오기

In [3]:
data = []
names = ['bitcoin.csv', 'repl.csv', 'etherium.csv']

for name in names:
    data.append(pd.read_csv(name))

# 결과 데이터를 저장할 데이터 프레임 생성

In [4]:
res = [] # 결과 데이터 저장

for ticker_data in data:
    res.append(ticker_data[['close', 'volume']])

res는 순서대로 bitcoin, reple, etherium의 기술지표를 저장할 리스트

In [5]:
time = data[0][['time']] # 시간을 저장할 데이터, 시간 데이터는 별도의 독립변수로 사용되지 않을 예정

In [6]:
res[0]

,close,volume
0,2996000.000,11139.871
1,3073000.000,8279.465
2,3119000.000,10731.677
3,3096000.000,9899.323
4,3083000.000,9472.610
...,...,...
4016,22320000.000,1666.680
4017,22403000.000,1587.818
4018,23004000.000,1675.061
4019,22790000.000,1505.955


In [7]:
class Indicator(object):
    def __init__(self,
                 data: List[pd.DataFrame],
                 res: List[pd.DataFrame]):
        self.data = data
        self.res = res
    
    def set_log_scale(self) -> None:
        '''
            transform close price to log scale close price
        '''
        for i in range(3):
            self.res[i].loc[:, 'close_log'] = np.log(self.data[i]['close'])
    
    def set_nvi(self) -> None:
        '''
            add nvi(negative volume index) to dataframe
        '''
        def nvi(res: pd.DataFrame, coin: pd.DataFrame) -> pd.DataFrame:
            nvi = [0] * len(coin)
            nvi[0] = 1
            close = coin['close'].copy()
            volume = coin['volume'].copy()
            
            for i in range(1, len(coin)):
                if volume[i] < volume[i-1]:
                    nvi[i] = nvi[i-1] + (close[i] - close[i-1]) * \
                            nvi[i-1] / close[i-1]
                else:
                    nvi[i] = nvi[i-1]
            res.loc[:, 'nvi'] = nvi
            return res
        
        for i in range(3):
            self.res[i] = nvi(self.res[i], self.data[i])
            
    def set_pvi(self) -> None:
        '''
            add pvi(positive volume index) to dataframe
        '''
        def pvi(res: pd.DataFrame, coin: pd.DataFrame) -> pd.DataFrame:
            pvi = [0] * len(coin)
            pvi[0] = 100
            volume = coin['volume']
            close = coin['close']
            
            for i in range(1, len(coin)):
                if volume[i] > volume[i-1]:
                    pvi[i] = pvi[i-1] + (close[i] - close[i-1] / \
                                close[i-1] * pvi[i-1])
                else:
                    pvi[i] = pvi[i-1]
            res.loc[:, 'pvi'] = pvi
            return res
        
        for i in range(3):
            self.res[i] = pvi(self.res[i], self.data[i]) 
    
    def set_ma(self) -> None:
        '''
            add ma(moving average) to dataframe
            It returns ma for 5, 10, 20, 60 intervals
            data's interval is 12hours
            '''
        def ma(res: pd.DataFrame, coin: pd.DataFrame) -> pd.DataFrame:
            days = [5, 10, 20, 60]
            close = coin['close']
            ma = pd.DataFrame()

            for day in days:
                ma.loc[:, 'ma_' + str(day)] = close.rolling(day).mean()
            res = pd.concat([res, ma], axis=1)
            
            return res
        
        for i in range(3):
            self.res[i] = ma(self.res[i], self.data[i])
        
    def set_rsi(self, period: int=14) -> None:
        '''
            add rsi(relative strength index) to dataframe
        '''
        def rsi(res: pd.DataFrame, coin: pd.DataFrame, period: int) -> pd.DataFrame:
            close = coin['close']
            
            U = np.where(close.diff(1) > 0, close.diff(1), 0)
            D = np.where(close.diff(1) < 0, close.diff(1) * (-1), 0)

            AU = pd.DataFrame(U).rolling(window=period, min_periods=period).mean()
            AD = pd.DataFrame(D).rolling(window=period, min_periods=period).mean()

            rsi = AU.div(AD+AU) * 100

            res.loc[:, 'rsi'] = rsi[0]
            
            return res
        
        for i in range(3):
            self.res[i] = rsi(self.res[i], self.data[i], period)
    
    def set_vpt(self) -> None:
        '''
            add vpt(volume price trend) to dataframe
        '''
        def vpt(res: pd.DataFrame, coin: pd.DataFrame) -> pd.DataFrame:
            vpt_list = [0] * len(coin)
            vpt_list[-1] = 2402.1359 # 최신 날짜 기준의 vpt
            volume = coin['volume']
            close = coin['close']

            for i in range(len(vpt_list) - 1, 0, -1):
                vpt_list[i-1] = vpt_list[i] - volume.iloc[i] * \
                                (close.iloc[i] - close.iloc[i-1]) / \
                                close.iloc[i-1]
            res.loc[:, 'vpt'] = vpt_list
            
            return res
            
        for i in range(3):
            self.res[i] = vpt(self.res[i], self.data[i])
            
    def set_obv(self) -> None:
        '''
            add obv(on-balance volume) to dataframe
        '''
        def obv(res: pd.DataFrame, coin: pd.DataFrame) -> pd.DataFrame:
            obv_list = [0] * len(coin)
            obv_list[0] = 68734.4525
            coin.loc[0, 'obv'] = 68734.4525
            volume = coin['volume']
            close = coin['close']
            
            for i in range(1, len(coin)):
                if close.iloc[i] > close.iloc[i-1]:
                    obv_list[i] = obv_list[i-1] + volume.iloc[i]
                elif close.iloc[i] == close.iloc[i-1]:
                    obv_list[i] = obv_list[i-1]
                else:
                    obv_list[i] = obv_list[i-1] - volume.iloc[i]
            res.loc[:, 'obv'] = obv_list
            
            return res
        
        for i in range(3):
            self.res[i] = obv(self.res[i], self.data[i])
            
    def set_std(self) -> None:
        '''
            add standard deviation to dataframe
            It returns std for 5, 10, 20, 60 intervals
            data's interval is 12hours
        '''
        def std(res: pd.DataFrame, coin: pd.DataFrame) -> pd.DataFrame:
            days = [5, 10, 20, 60]
            close = coin['close']
            deviation = pd.DataFrame()

            for day in days:
                deviation.loc[:, 'std_' + str(day)] = coin['close'].rolling(day).mean()


            res = pd.concat([res, deviation], axis=1)
            
            return res
        
        for i in range(3):
            self.res[i] = std(self.res[i], self.data[i])
        
    def set_mfi(self, period: int=14) -> None:
        '''
            set mfi(money flow index) to dataframe
            can set periods
        '''
        def mfi(res: pd.DataFrame, coin: pd.DataFrame,
               period: int) -> pd.DataFrame:
            close = coin['close']
            high = coin['high']
            low = coin['low']
            volume = coin['volume']
            
            typical_price = (close + high + low) / 3
            money_flow = typical_price * volume
            positive_flow = []
            negative_flow = []
            
            for i in range(1, len(typical_price)):
                if typical_price[i] > typical_price[i-1]:
                    positive_flow.append(money_flow[i-1])
                    negative_flow.append(0)
                elif typical_price[i] < typical_price[i-1]:
                    positive_flow.append(0)
                    negative_flow.append(money_flow[i-1])
                else:
                    positive_flow.append(0)
                    negative_flow.append(0)
            
            positive_mf = []
            negative_mf = []
            
            for i in range(period-1, len(positive_flow)):
                positive_mf.append(sum(positive_flow[i+1-period:i+1]))
            
            for i in range(period-1, len(negative_flow)):
                negative_mf.append(sum(negative_flow[i+1-period:i+1]))
                
            mfi = 100 * (np.array(positive_mf) / \
                        (np.array(positive_mf) + np.array(negative_mf)))
            res.loc[:, 'mfi'] = np.r_['0, 1', np.full(period, np.nan), mfi]
            
            return res
        
        for i in range(3):
            self.res[i] = mfi(self.res[i], self.data[i], period)
            
    def set_ema(self) -> None:
        '''
            set ema(exponential moving average) to dataframe
            It returns ema for 5, 10, 20, 60 intervals
            data's interval is 12hours
        '''
        def ema(res: pd.DataFrame, coin: pd.DataFrame) -> pd.DataFrame:
            days = [5, 10, 20, 60]
            close = coin['close']
            
            for day in days:
                ema = close.ewm(span=day, adjust=False).mean()
                res.loc[:, 'ema_' + str(day)] = ema
            return res
        
        for i in range(3):
            self.res[i] = ema(self.res[i], self.data[i])
            
    def set_fi(self, period: int=14) -> None:
        '''
            set fi(force index) to dataframe
        '''
        def fi(res: pd.DataFrame, coin: pd.DataFrame,
              period: int) -> pd.DataFrame:
            close = coin['close']
            volume = coin['volume']
            
            fi = pd.Series(close.diff(period) * volume, name='fi')
            res.loc[:, 'fi'] = fi
            
            return res
        
        for i in range(3):
            self.res[i] = fi(self.res[i], self.data[i], period)
            
    def set_bb(self, period: int=20, k: int=2) -> None:
        '''
            set bb(bollinger band) to dataframe
        '''
        def bb(res: pd.DataFrame, coin: pd.DataFrame,
              period: int, k: int) -> pd.DataFrame:
            x = coin['close']
            mbb = x.rolling(period).mean()
            ubb = mbb + k * x.rolling(period).std()
            lbb = mbb - k * x.rolling(period).std()
            
            bollinger_band= pd.DataFrame()
            bollinger_band['ubb'] = ubb
            bollinger_band['mbb'] = mbb
            bollinger_band['lbb'] = lbb
            
            res = pd.concat([res, bollinger_band], axis=1)
            
            return res
        
        for i in range(3):
            self.res[i] = bb(self.res[i], self.data[i], period, k)
            
    def set_indicators(self) -> None:
        self.set_log_scale()
        self.set_nvi()
        self.set_pvi()
        self.set_ma()
        self.set_rsi()
        self.set_vpt()
        self.set_obv()
        self.set_std()
        self.set_mfi()
        self.set_ema()
        self.set_fi()
        self.set_bb()
            
    def get_res(self) -> List[pd.DataFrame]:
        '''
            return Indicator(object)'s result
        '''
        for i in range(3):
            self.res[i].dropna(inplace=True)
            self.res[i].reset_index(drop=True, inplace=True)
        return self.res

In [8]:
indicator = Indicator(data, res)

In [9]:
indicator.set_indicators()

In [10]:
res = indicator.get_res()

## 기술지표 요약

In [11]:
res[0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3962 entries, 0 to 3961
Data columns (total 25 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   close      3962 non-null   float64
 1   volume     3962 non-null   float64
 2   close_log  3962 non-null   float64
 3   nvi        3962 non-null   float64
 4   pvi        3962 non-null   float64
 5   ma_5       3962 non-null   float64
 6   ma_10      3962 non-null   float64
 7   ma_20      3962 non-null   float64
 8   ma_60      3962 non-null   float64
 9   rsi        3962 non-null   float64
 10  vpt        3962 non-null   float64
 11  obv        3962 non-null   float64
 12  std_5      3962 non-null   float64
 13  std_10     3962 non-null   float64
 14  std_20     3962 non-null   float64
 15  std_60     3962 non-null   float64
 16  mfi        3962 non-null   float64
 17  ema_5      3962 non-null   float64
 18  ema_10     3962 non-null   float64
 19  ema_20     3962 non-null   float64
 20  ema_60  

In [12]:
res[1].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3962 entries, 0 to 3961
Data columns (total 25 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   close      3962 non-null   float64
 1   volume     3962 non-null   float64
 2   close_log  3962 non-null   float64
 3   nvi        3962 non-null   float64
 4   pvi        3962 non-null   float64
 5   ma_5       3962 non-null   float64
 6   ma_10      3962 non-null   float64
 7   ma_20      3962 non-null   float64
 8   ma_60      3962 non-null   float64
 9   rsi        3962 non-null   float64
 10  vpt        3962 non-null   float64
 11  obv        3962 non-null   float64
 12  std_5      3962 non-null   float64
 13  std_10     3962 non-null   float64
 14  std_20     3962 non-null   float64
 15  std_60     3962 non-null   float64
 16  mfi        3962 non-null   float64
 17  ema_5      3962 non-null   float64
 18  ema_10     3962 non-null   float64
 19  ema_20     3962 non-null   float64
 20  ema_60  

In [13]:
res[2].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3962 entries, 0 to 3961
Data columns (total 25 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   close      3962 non-null   float64
 1   volume     3962 non-null   float64
 2   close_log  3962 non-null   float64
 3   nvi        3962 non-null   float64
 4   pvi        3962 non-null   float64
 5   ma_5       3962 non-null   float64
 6   ma_10      3962 non-null   float64
 7   ma_20      3962 non-null   float64
 8   ma_60      3962 non-null   float64
 9   rsi        3962 non-null   float64
 10  vpt        3962 non-null   float64
 11  obv        3962 non-null   float64
 12  std_5      3962 non-null   float64
 13  std_10     3962 non-null   float64
 14  std_20     3962 non-null   float64
 15  std_60     3962 non-null   float64
 16  mfi        3962 non-null   float64
 17  ema_5      3962 non-null   float64
 18  ema_10     3962 non-null   float64
 19  ema_20     3962 non-null   float64
 20  ema_60  

## 비트코인 기술지표

In [14]:
res[0]

,close,volume,close_log,nvi,pvi,ma_5,ma_10,ma_20,ma_60,rsi,...,std_60,mfi,ema_5,ema_10,ema_20,ema_60,fi,ubb,mbb,lbb
0,3091000.000,5388.618,14.944,1.163,3063000.000,3084200.000,3170200.000,3289500.000,3215300.000,24.735,...,3215300.000,53.118,3101463.544,3154992.252,3207830.411,3195371.939,-2058452065.648,3610062.335,3289500.000,2968937.665
1,3087000.000,6582.365,14.943,1.163,3087000.000,3088600.000,3137500.000,3279000.000,3216816.667,25.546,...,3216816.667,54.028,3096642.362,3142630.024,3196322.753,3191818.761,-2356486583.797,3612042.118,3279000.000,2945957.882
2,3071000.000,3959.027,14.938,1.157,3087000.000,3085200.000,3106200.000,3262700.000,3216783.333,25.304,...,3216783.333,52.649,3088094.908,3129606.384,3184387.253,3187857.490,-1445044879.170,3603251.318,3262700.000,2922148.682
3,3050000.000,5515.104,14.931,1.157,3050000.000,3072400.000,3086100.000,3243350.000,3215633.333,26.045,...,3215633.333,53.324,3075396.605,3115132.496,3171588.467,3183337.572,-1897195628.947,3586172.510,3243350.000,2900527.490
4,3003000.000,4081.218,14.915,1.140,3050000.000,3060400.000,3063700.000,3223000.000,3214083.333,25.442,...,3214083.333,54.476,3051264.404,3094744.769,3155532.422,3177424.865,-1473319810.971,3572425.995,3223000.000,2873574.005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3957,22320000.000,1666.680,16.921,19.408,22320000.000,22490200.000,22593100.000,22691850.000,24935000.000,46.624,...,24935000.000,53.458,22458484.655,22563676.674,22779918.772,24315271.557,-211668390.658,23160372.100,22691850.000,22223327.900
3958,22403000.000,1587.818,16.925,19.480,22320000.000,22430800.000,22571300.000,22662550.000,24822750.000,48.006,...,24822750.000,53.336,22439989.770,22534462.733,22744021.746,24252574.129,-122261984.913,23126095.365,22662550.000,22199004.635
3959,23004000.000,1675.061,16.951,19.480,23004000.000,22496800.000,22612900.000,22660250.000,24726633.333,51.684,...,24726633.333,43.927,22627993.180,22619833.145,22768781.580,24211637.272,117254288.642,23116093.406,22660250.000,22204406.594
3960,22790000.000,1505.955,16.942,19.299,23004000.000,22581400.000,22618800.000,22651250.000,24629066.667,47.797,...,24629066.667,45.856,22681995.453,22650772.573,22770802.382,24165026.214,-149089515.332,23088053.641,22651250.000,22214446.359


## 리플 기술지표

In [15]:
res[1]

,close,volume,close_log,nvi,pvi,ma_5,ma_10,ma_20,ma_60,rsi,...,std_60,mfi,ema_5,ema_10,ema_20,ema_60,fi,ubb,mbb,lbb
0,314.000,18455143.631,5.749,0.800,312.000,316.400,327.300,348.750,341.450,36.000,...,341.450,35.979,317.912,326.085,333.943,336.766,-775116032.511,405.519,348.750,291.981
1,314.000,42393949.971,5.749,0.800,314.000,316.800,322.600,344.300,341.183,31.915,...,341.183,33.170,316.608,323.888,332.044,336.020,-2162091448.530,396.968,344.300,291.632
2,310.000,25645693.112,5.737,0.790,314.000,315.000,318.200,340.300,340.383,21.260,...,340.383,25.976,314.405,321.363,329.945,335.167,-1872135597.166,390.445,340.300,290.155
3,304.000,33557047.433,5.717,0.790,304.000,310.800,314.500,337.150,335.950,23.894,...,335.950,28.594,310.937,318.206,327.474,334.145,-1979865798.547,388.141,337.150,286.159
4,302.000,16835866.677,5.710,0.784,304.000,308.800,311.300,333.950,333.750,22.807,...,333.750,33.140,307.958,315.259,325.048,333.091,-1043823733.963,385.349,333.950,282.551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3957,528.800,51434464.678,6.271,0.670,528.800,536.300,542.760,529.410,564.648,59.820,...,564.648,75.668,533.909,535.680,533.931,559.041,1121271329.991,568.803,529.410,490.017
3958,534.600,44094548.049,6.282,0.677,528.800,533.220,542.900,529.955,562.715,59.269,...,562.715,74.823,534.140,535.484,533.995,558.240,895119325.396,569.317,529.955,490.593
3959,548.100,46041013.658,6.306,0.677,548.100,532.140,543.730,530.330,561.270,61.385,...,561.270,72.274,538.793,537.778,535.338,557.907,1210878659.195,570.258,530.330,490.402
3960,543.400,35095065.973,6.298,0.671,548.100,536.300,542.550,531.550,559.723,59.281,...,559.723,72.580,540.329,538.800,536.106,557.431,779110464.593,571.511,531.550,491.589


## 이더리움 기술지표

In [16]:
res[2]

,close,volume,close_log,nvi,pvi,ma_5,ma_10,ma_20,ma_60,rsi,...,std_60,mfi,ema_5,ema_10,ema_20,ema_60,fi,ubb,mbb,lbb
0,357000.000,260618.673,12.785,1.107,358500.000,358320.000,353255.000,390420.000,371854.167,38.333,...,371854.167,37.598,357884.584,363117.353,378202.394,374569.544,-16418976402.385,482972.878,390420.000,297867.122
1,363250.000,333645.944,12.803,1.107,363250.000,366010.000,349980.000,385677.500,372734.167,39.586,...,372734.167,39.472,359673.056,363141.471,376778.356,374198.411,-19151277168.693,473212.717,385677.500,298142.283
2,350950.000,198054.732,12.768,1.070,363250.000,363200.000,346720.000,380595.000,373530.000,38.074,...,373530.000,37.899,356765.371,360924.840,374318.513,373436.168,-13537040913.880,463448.338,380595.000,297741.662
3,333000.000,432084.274,12.716,1.070,333000.000,352540.000,345915.000,375200.000,374230.833,37.357,...,374230.833,37.081,348843.581,355847.596,370383.416,372110.392,-31909423649.937,455533.096,375200.000,294866.904
4,324000.000,178421.284,12.688,1.041,333000.000,345640.000,344625.000,369807.500,374734.167,36.586,...,374734.167,35.740,340562.387,350057.124,365965.948,370533.002,-14273702702.068,448593.441,369807.500,291021.559
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3957,1623000.000,7986.891,14.300,37.953,1623000.000,1636000.000,1641200.000,1629050.000,1847700.000,61.925,...,1847700.000,58.454,1629895.529,1634740.680,1649294.657,1769473.931,455252796.796,1700646.751,1629050.000,1557453.249
3958,1652000.000,9103.184,14.317,37.953,1652000.000,1632800.000,1643100.000,1628300.000,1837800.000,66.292,...,1837800.000,64.978,1637263.686,1637878.738,1649552.309,1765622.327,791976994.975,1698524.002,1628300.000,1558075.998
3959,1723000.000,11768.645,14.360,37.953,1723000.000,1645000.000,1653200.000,1630200.000,1829666.667,68.641,...,1829666.667,60.789,1665842.457,1653355.331,1656547.327,1764224.873,1259245013.171,1708477.778,1630200.000,1551922.222
3960,1714000.000,6872.623,14.354,37.755,1723000.000,1664800.000,1660700.000,1634400.000,1821700.000,66.096,...,1821700.000,62.130,1681894.972,1664381.635,1662019.010,1762578.156,646026588.099,1721184.427,1634400.000,1547615.573


## 시간 데이터 저장

In [17]:
time.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4021 entries, 0 to 4020
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   time    4021 non-null   object
dtypes: object(1)
memory usage: 31.5+ KB


In [18]:
remove_row = len(time) - len(res[0])
time = time.drop(time.head(59).index)
time = time.reset_index(drop=True)

In [19]:
time

,time
0,2017-06-30 00:00:00
1,2017-06-30 12:00:00
2,2017-07-01 00:00:00
3,2017-07-01 12:00:00
4,2017-07-02 00:00:00
...,...
3957,2022-11-29 00:00:00
3958,2022-11-29 12:00:00
3959,2022-11-30 00:00:00
3960,2022-11-30 12:00:00


In [20]:
time.to_csv('time.csv', index=False)

## 기술지표 및 시간 데이터 저장

In [21]:
res[0].to_csv('bitcoin_indicator.csv', index=False)
res[1].to_csv('repl_indicator.csv', index=False)
res[2].to_csv('etherium_indicator.csv', index=False)

# 기술지표 참조

## Log scale
$$
log\; close\; price = log(close)
$$

## NVI(Negative Volume Index)
$$
\begin{cases}
NVI = NVI_{prev} + \frac{close - close_{prev}} {close_prev} \times NVI_{prev}\quad, 
\;volume_{today} < volume_{yesterday}\\
0\quad \text{, otherwise}\\
\end{cases}
$$

## PVI(Positive Volume Index)
$$
\begin{cases}
PVI = PVI_{prev} + \frac{close}{close_prev} \times 100\quad ,\; volume_{today} > volume_{yesterday}\\
0\quad ,\; \text{otherwise}
\end{cases}
$$

## MA(Moving Average)
$$
MA_k = \frac{1}{k} \sum_{i=n-k+1}^{n}p_i
$$

## RSI(Relative Strength Index)
$$
\begin{align}
&case1.\; \text{the close being higher than the previous close}\\
&\quad U = close_{now} - close{prev}\\
&\quad D = 0\\
&\\
&case2.\; \text{the close being lower than the previous period's close}\\
&\quad U = 0\\
&\quad D = close_{prev} - close_{now}\\
&\\
&RS = \frac{SMMA(U, n)}{SMMA(D, n)}\\
&RSI = 100 - \frac{100}{1+RS}
\end{align}
$$

## VPT(Volume Price Trend)
$$
VPT = VPT_{prev} + volume \times \frac{close_{today} - close_{prev}} {close_{prev}}
$$

## OBV(On-balance Volume)
$$
OBV = OBV_{prev} +
\begin{cases}
    volume \quad \text{if close > }close_{prev}\\
    0 \quad \text{if close = }close_{prev}\\
    -volume \quad \text{if close < }close_{prev}
\end{cases}
$$

## STD(Standard Deviation)
$$
\sigma = \sqrt{\frac{1}{N}\sum_{i=1}^{N}(x_i - \mu) ^ 2}
$$

## MFI(Money Flow Index)
$$
\begin{align}
    &typical\; price = \frac{high + low + close} {3}\\
    &money\; flow = typical\; price \times volume\\
    &\begin{cases}
    &    \text{positive money flow + money flow, if typical price > } typical\; price_{prev}\\
    &    \text{negative money flow + money flow, if typical price < } typical\; price_{prev}
    &\end{cases}\\
    &money\; ratio = \frac{positive\; money\; flow} {negative\; money\; flow}\\
    &MFI = 100 - \frac{100} {1 + money\; ratio}
\end{align}
$$

## EMA(Exponential Moving Average)
$$
    EMA = price \times \frac{smoothing}{1 + days} + EMA_{prev} \times (1 - \frac{smoothing}{1 + days})
$$

## FI(Force Index)
$$
    FI = close - close_{prev} \times volume
$$

## BB(Bollinger Band)
$$
\begin{align}
    &\%b = \frac {(last - lowerBB)} {upperBB - lowerBB}\\
    &Bandwidth = \frac {upperBB - lowerBB} {middleBB}
\end{align}
$$